# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Program: Computer Systems Engineering  </center>
---
### <center> **Spring 2025** </center>
---

**Lab 07**: Structured Streaming with Files

**Date**: 31/03/2025

**Team Name**: Par De Dos

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ParDeDosLogsStreaming") \
    .master("spark://a61219ce2752:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/08 01:01:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
log_dir="/home/jovyan/notebooks/data/structured_streaming_log_files/"

logs_lines = spark \
                .readStream \
                .format("text") \
                .option("maxFilesPerTrigger", 5) \
                .load(log_dir)

In [4]:
from pyspark.sql.functions import split, col

logs_df = logs_lines.select(split(col("value"), r"\s*\|\s*").alias("log_array"))

logs_df = logs_df.withColumn("date_time", col("log_array")[0])
logs_df = logs_df.withColumn("log_level", col("log_array")[1])
logs_df = logs_df.withColumn("message", col("log_array")[2])
logs_df = logs_df.withColumn("node", col("log_array")[3])


logs_df.printSchema()

root
 |-- log_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- date_time: string (nullable = true)
 |-- log_level: string (nullable = true)
 |-- message: string (nullable = true)
 |-- node: string (nullable = true)



In [5]:
logs_df = logs_df.filter(logs_df.log_level == "ERROR")

In [ ]:
from team_ParDeDos.query_listeners import QueryListener as QL

spark.streams.addListener(QL())

query = logs_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='5 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(30)

25/04/08 01:03:17 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ebccec16-b111-4f8a-b5d3-f84c4e8c6a15. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/08 01:03:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Query started: a39a5958-8f3f-4f23-a4b8-1081a63d9366


-------------------------------------------
Batch: 0
-------------------------------------------
+---------------------------------------------------------------------+-------------------+---------+------------------------+-------------+
|log_array                                                            |date_time          |log_level|message                 |node         |
+---------------------------------------------------------------------+-------------------+---------+------------------------+-------------+
|[2025-04-08 01:03:12, ERROR, Database connection lost, server-node-2]|2025-04-08 01:03:12|ERROR    |Database connection lost|server-node-2|
|[2025-04-08 01:03:12, ERROR, Imminent Explosion, server-node-3]      |2025-04-08 01:03:12|ERROR    |Imminent Explosion      |server-node-3|
+---------------------------------------------------------------------+-------------------+---------+------------------------+-------------+



25/04/08 01:03:24 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 6645 milliseconds


Query made progress: {
  "id" : "a39a5958-8f3f-4f23-a4b8-1081a63d9366",
  "runId" : "2aab5cef-1a55-4918-bb07-8f8a35a29072",
  "name" : null,
  "timestamp" : "2025-04-08T01:03:17.477Z",
  "batchId" : 0,
  "numInputRows" : 10,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 1.5167602002123464,
  "durationMs" : {
    "addBatch" : 5373,
    "commitOffsets" : 91,
    "getBatch" : 125,
    "latestOffset" : 93,
    "queryPlanning" : 760,
    "triggerExecution" : 6592,
    "walCommit" : 113
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "FileStreamSource[file:/home/jovyan/notebooks/data/structured_streaming_log_files]",
    "startOffset" : null,
    "endOffset" : {
      "logOffset" : 0
    },
    "latestOffset" : null,
    "numInputRows" : 10,
    "inputRowsPerSecond" : 0.0,
    "processedRowsPerSecond" : 1.5167602002123464
  } ],
  "sink" : {
    "description" : "org.apache.spark.sql.execution.streaming.ConsoleTable$@3c20d872",
    "numOutputRows" : 2
  }
}
-

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------------------------------------------------------------------+-------------------+---------+-----------------------------------+-------------+
|log_array                                                                       |date_time          |log_level|message                            |node         |
+--------------------------------------------------------------------------------+-------------------+---------+-----------------------------------+-------------+
|[2025-04-08 01:03:27, ERROR, Unauthorized login attempt detected, server-node-3]|2025-04-08 01:03:27|ERROR    |Unauthorized login attempt detected|server-node-3|
+--------------------------------------------------------------------------------+-------------------+---------+-----------------------------------+-------------+

Query made progress: {
  "id" : "a39a5958-8f3f-4f23-a4b8-1081a63d9366",
  "runId" : "2a

In [8]:
query.stop()

In [7]:
sc.stop()